# [Retrieval-based-Voice-Conversion-WebUI](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI) Training notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb)

In [ ]:
# @title 【MONITOREO AVANZADO de GPU】
# @markdown ---
# @markdown 🔍 Información detallada sobre tu GPU para optimizar RVC

import torch
from pprint import pprint

# @markdown ---
# @markdown 1. Información básica del sistema
print("🖥️ Información del sistema:")
!cat /etc/os-release | grep PRETTY_NAME
!python --version
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# @markdown ---
# @markdown 2. Estado detallado de la GPU
print("\n🎮 Estado NVIDIA-SMI:")
!nvidia-smi --query-gpu=name,driver_version,memory.total,memory.used,memory.free,temperature.gpu,utilization.gpu,utilization.memory --format=csv

# @markdown ---
# @markdown 3. Configuración óptima para RVC
print("\n🔧 Recomendaciones para RVC:")
gpu_mem = !nvidia-smi --query-gpu=memory.total --format=csv,noheader,nounits
gpu_mem = int(gpu_mem[0])

if gpu_mem >= 16000:
    print("✅ GPU potente detectada (>16GB):")
    print("- Puedes usar batch_size=8-16")
    print("- Habilita --fp16 para inferencia")
elif gpu_mem >= 8000:
    print("⚠️ GPU moderada detectada (8-16GB):")
    print("- Usa batch_size=4-8")
    print("- Considera --fp16 para ahorrar memoria")
else:
    print("❌ GPU limitada detectada (<8GB):")
    print("- Usa batch_size=2-4")
    print("- Desactiva --fp16")
    print("- Considera Colab Pro")

# @markdown ---
# @markdown 4. Monitorización en tiempo real (opcional)
monitor = False  # @param {type:"boolean"}
if monitor:
    print("\n📊 Monitorización continua (Ctrl+C para detener):")
    !nvidia-smi -l 1
else:
    print("\n💡 Para monitorización continua ejecuta: !nvidia-smi -l 1")

# @markdown ---
# @markdown 5. Información adicional de PyTorch
if torch.cuda.is_available():
    print("\n⚙️ Configuración actual de PyTorch:")
    print(f"GPU actual: {torch.cuda.current_device()}")
    print(f"Nombre GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria reservada: {torch.cuda.memory_reserved(0)/1024**2:.2f} MB")
    print(f"Memoria usada: {torch.cuda.memory_allocated(0)/1024**2:.2f} MB")
else:
    print("\n❌ No se detectó GPU compatible")

In [ ]:
# @title 【MONTAJE SEGURO de Google Drive】
# @markdown ---
# @markdown 🔐 Montaje seguro de Google Drive con verificación de permisos

import os
from google.colab import drive
from IPython.display import clear_output

# @markdown ---
# @markdown 1. Configuración de montaje
MOUNT_POINT = "/content/drive"  # @param {type:"string"}
FORCE_REMOUNT = False  # @param {type:"boolean"}

# @markdown ---
# @markdown 2. Verificación de montajes existentes
if os.path.ismount(MOUNT_POINT):
    if FORCE_REMOUNT:
        print(f"🔌 Desmontando {MOUNT_POINT} existente...")
        !fusermount -u {MOUNT_POINT}
    else:
        print(f"✅ Drive ya montado en {MOUNT_POINT}")
        !df -h | grep drive
        exit()

# @markdown ---
# @markdown 3. Montaje con manejo de errores
try:
    print(f"🔌 Conectando a Google Drive en {MOUNT_POINT}...")
    drive.mount(MOUNT_POINT, force_remount=FORCE_REMOUNT)

    # Verificación post-montaje
    if not os.path.ismount(MOUNT_POINT):
        raise RuntimeError("El montaje falló silenciosamente")

    clear_output()
    print(f"✅ Google Drive montado correctamente en {MOUNT_POINT}")
    print("\n📂 Contenido raíz de Drive:")
    !ls -lha "{MOUNT_POINT}/MyDrive/"

except Exception as e:
    print(f"❌ Error durante el montaje: {str(e)}")
    print("\n🔧 Soluciones recomendadas:")
    print("1. Verifica tu conexión a internet")
    print("2. Asegúrate de estar logueado en la cuenta correcta")
    print("3. Prueba reiniciando el entorno (Runtime > Restart runtime)")
    print("4. Intenta con: drive.mount('/content/drive', force_remount=True)")

# @markdown ---
# @markdown 4. Acciones post-montaje recomendadas
print("\n💡 Acciones útiles después del montaje:")
print(f"- Crear enlace simbólico: !ln -s '{MOUNT_POINT}/MyDrive/RVC_Data' '/content/RVC'")
print("- Navegar a tu carpeta: %cd '/content/drive/MyDrive/RVC_Projects'")
print("- Verificar espacio: !df -h {MOUNT_POINT}")

# @markdown ---
# @markdown 5. Configuración de permisos (opcional)
set_permissions = False  # @param {type:"boolean"}
if set_permissions:
    print("\n🔒 Ajustando permisos...")
    !chmod -R 755 "{MOUNT_POINT}/MyDrive"
    print("Permisos actualizados a 755 (lectura/ejecución para todos)")

In [ ]:
# @title 【INSTALACIÓN COMPLETA de Dependencias】
# @markdown ---
# @markdown 🔧 Instala todos los requisitos para RVC con control de versiones

import subprocess
import importlib

# @markdown ---
# @markdown 1. Dependencias del sistema
print("🛠️ Instalando dependencias del sistema...")
system_deps = [
    "build-essential",
    "python3-dev",
    "ffmpeg"
]

for dep in system_deps:
    try:
        !apt-get install -y -qq {dep}
        print(f"✅ {dep} instalado")
    except:
        print(f"⚠️ Error instalando {dep}")

# @markdown ---
# @markdown 2. Actualización de herramientas básicas
print("\n🔄 Actualizando herramientas Python...")
!python -m pip install --upgrade --quiet pip setuptools wheel

# @markdown ---
# @markdown 3. Paquetes principales (con versiones específicas)
core_packages = {
    "faiss-cpu": "1.7.2",
    "fairseq": "",
    "gradio": "3.14.0",
    "ffmpeg-python": "",
    "praat-parselmouth": "",
    "pyworld": "",
    "numpy": "1.23.5",
    "numba": "0.56.4",
    "librosa": "0.9.2",
    "torchaudio": "0.13.1+cu116",
    "torch": "1.12.1+cu116"
}

print("\n📦 Instalando paquetes principales:")
for pkg, ver in core_packages.items():
    version_spec = f"=={ver}" if ver else ""
    try:
        !pip install --quiet {pkg}{version_spec}

        # Verificación
        installed = subprocess.getoutput(f"pip show {pkg} | grep Version")
        print(f"✅ {pkg} {installed}")
    except:
        print(f"❌ Error instalando {pkg}{version_spec}")

# @markdown ---
# @markdown 4. Verificación final
print("\n🔍 Versiones instaladas clave:")
key_packages = ["numpy", "librosa", "torch", "gradio"]
for pkg in key_packages:
    try:
        version = importlib.import_module(pkg).__version__
        print(f"{pkg:>15}: {version}")
    except:
        print(f"{pkg:>15}: NO INSTALADO")

# @markdown ---
# @markdown 5. Solución de problemas comunes
print("\n⚠️ Si hay errores con Numba/Librosa:")
print("!pip uninstall -y numba librosa numpy")
print("!pip install numba==0.56.4 librosa==0.9.2 numpy==1.23.5")

In [ ]:
# @title 【CLONACIÓN SEGURA del Repositorio RVC】
# @markdown ---
# @markdown 🛠️ Clona el repositorio principal de RVC y prepara la estructura de directorios

import os
import shutil
from datetime import datetime

# @markdown ---
# @markdown 1. Configuración de clonación
REPO_URL = "https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI"  # @param ["https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI", "https://github.com/fumiama/Retrieval-based-Voice-Conversion-WebUI"]
BRANCH = "main"  # @param ["main", "stable", "v2"] {type:"string"}
TARGET_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI"

# @markdown ---
# @markdown 2. Limpieza previa (si existe instalación anterior)
clean_previous = True  # @param {type:"boolean"}
if clean_previous and os.path.exists(TARGET_DIR):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_dir = f"/content/RVC_backup_{timestamp}"

    print(f"🧹 Moviendo instalación anterior a {backup_dir}...")
    shutil.move(TARGET_DIR, backup_dir)

# @markdown ---
# @markdown 3. Clonación optimizada
try:
    print(f"⬇️ Clonando repositorio {REPO_URL} (rama: {BRANCH})...")
    !git clone --depth=1 -b {BRANCH} {REPO_URL} {TARGET_DIR}

    # @markdown ---
    # @markdown 4. Verificación de clonación
    if not os.path.exists(f"{TARGET_DIR}/.git"):
        raise RuntimeError("La clonación falló - directorio .git no encontrado")

    print("\n✅ Repositorio clonado correctamente")
    print(f"📦 Último commit: {subprocess.getoutput(f'git -C {TARGET_DIR} log -1 --pretty=%B').strip()}")

    # @markdown ---
    # @markdown 5. Preparación de directorios esenciales
    essential_dirs = [
        "pretrained",
        "uvr5_weights",
        "logs",
        "assets/weights"
    ]

    print("\n📂 Creando estructura de directorios:")
    for directory in essential_dirs:
        dir_path = os.path.join(TARGET_DIR, directory)
        os.makedirs(dir_path, exist_ok=True)
        print(f" - {dir_path}")

    # @markdown ---
    # @markdown 6. Instalación de dependencias básicas
    print("\n🔧 Instalando requisitos básicos...")
    !pip install -q -r "{TARGET_DIR}/requirements.txt"

except Exception as e:
    print(f"\n❌ Error durante clonación: {str(e)}")
    print("\n🔧 Soluciones alternativas:")
    print(f"1. Descarga manual: !wget {REPO_URL}/archive/refs/heads/{BRANCH}.zip")
    print("2. Usar mirror: !git clone https://ghproxy.com/{REPO_URL}")
    print("3. Verificar conexión a GitHub")

# @markdown ---
# @markdown 7. Verificación final
print("\n🔍 Estructura resultante:")
!ls -lh {TARGET_DIR}

In [ ]:
# @title 【ACTUALIZACIÓN SEGURA del Repositorio RVC】
# @markdown ---
# @markdown 🔄 Actualiza el código del proyecto desde GitHub
# @markdown ⚠️ Solo ejecutar si hay actualizaciones críticas anunciadas

import os
import shutil
from datetime import datetime

# @markdown ---
# @markdown 1. Configuración inicial
REPO_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI"
BACKUP_DIR = f"/content/RVC_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# @markdown ---
# @markdown 2. Verificación del estado actual
print("🔍 Estado actual del repositorio:")
!git -C {REPO_DIR} status
!git -C {REPO_DIR} log -n 1 --pretty=format:"%h - %s (%cr)"

# @markdown ---
# @markdown 3. Crear backup (opcional pero recomendado)
create_backup = True  # @param {type:"boolean"}
if create_backup:
    print(f"\n💾 Creando backup en {BACKUP_DIR}...")
    os.makedirs(BACKUP_DIR, exist_ok=True)
    !cp -r {REPO_DIR}/* {BACKUP_DIR}/
    print("✅ Backup completado")

# @markdown ---
# @markdown 4. Actualización controlada
try:
    print("\n🔄 Actualizando repositorio...")
    !git -C {REPO_DIR} pull --rebase

    # Verificar conflicto
    if "conflict" in subprocess.getoutput(f"git -C {REPO_DIR} status"):
        raise Exception("Conflicto de merge detectado")

    print("\n📦 Actualizando dependencias...")
    !pip install -r {REPO_DIR}/requirements.txt --upgrade

    print("\n✅ Actualización completada con éxito")
    print("📄 Cambios recientes:")
    !git -C {REPO_DIR} log -n 3 --pretty=format:"%h - %s (%cr)"

except Exception as e:
    print(f"\n❌ Error durante actualización: {str(e)}")

    if create_backup:
        print("\n🛑 Restaurando desde backup...")
        !rm -rf {REPO_DIR}/*
        !cp -r {BACKUP_DIR}/* {REPO_DIR}/
        print("✅ Sistema restaurado desde backup")

    print("\n🔧 Soluciones recomendadas:")
    print("1. Clonar manualmente: !git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI")
    print("2. Restaurar notebook original")

# @markdown ---
# @markdown 5. Verificación post-actualización
print("\n🔍 Estado final:")
!git -C {REPO_DIR} status

In [ ]:
# @title 【INSTALACIÓN OPTIMIZADA de aria2】
# @markdown ---
# @markdown 🔍 Instala y configura aria2 para descargas aceleradas

import os
import subprocess

# @markdown ---
# @markdown 1. Instalación con verificación
try:
    print("🔄 Actualizando paquetes...")
    !apt-get update -qq

    print("⬇️ Instalando aria2...")
    install_cmd = "apt -y install -qq aria2"
    result = subprocess.run(install_cmd, shell=True, capture_output=True, text=True)

    if result.returncode == 0:
        print("✅ aria2 instalado correctamente")

        # Verificación de versión
        version = subprocess.getoutput("aria2c --version | head -n 1")
        print(f"ℹ️ Versión instalada: {version}")

        # Configuración óptima
        config_dir = "/etc/aria2"
        if not os.path.exists(config_dir):
            os.makedirs(config_dir)

        with open(f"{config_dir}/aria2.conf", "w") as f:
            f.write("""## Configuración optimizada para Colab ##
max-concurrent-downloads=10
split=16
max-connection-per-server=16
min-split-size=1M
continue=true
""")
        print(f"⚙️ Configuración guardada en {config_dir}/aria2.conf")
    else:
        print(f"❌ Error durante instalación: {result.stderr}")
        print("Intentando con método alternativo...")
        !apt --fix-broken install -y
        !apt -y install aria2

except Exception as e:
    print(f"❌ Error crítico: {str(e)}")
    print("Soluciones alternativas:")
    print("1. Usar wget/curl en lugar de aria2")
    print("2. Reiniciar el entorno de ejecución (Runtime -> Restart runtime)")

# @markdown ---
# @markdown 2. Prueba de funcionamiento
print("\n🧪 Probando descarga de prueba...")
test_url = "https://github.com/aria2/aria2/releases/download/release-1.36.0/aria2-1.36.0.tar.gz"
!aria2c --console-log-level=warn -c -x 8 -s 8 -k 1M {test_url} -d /tmp -o test_download.tgz

if os.path.exists("/tmp/test_download.tgz"):
    print("✅ Prueba exitosa. aria2 funciona correctamente")
    !rm -f "/tmp/test_download.tgz"
else:
    print("❌ La prueba de descarga falló")

# @markdown ---
# @markdown 3. Configuración recomendada para RVC
print("\n🔧 Configuración óptima para descargas RVC:")
print("""
!aria2c --console-log-level=error \\
        -c -x 16 -s 16 \\
        -k 1M \\
        --file-allocation=none \\
        --summary-interval=0 \\
        [URL] -d [DIRECTORIO] -o [ARCHIVO]
""")

In [ ]:
# @title 【DESCARGA DE MODELOS BASE - Versión Mejorada】
# @markdown ---
# @markdown 🔍 Descarga todos los modelos base necesarios para RVC

import os
import hashlib
import concurrent.futures
from tqdm.notebook import tqdm

# @markdown ---
# @markdown 1. Configuración de modelos
MODELS = [
    # Discriminadores
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth", "name": "D32k.pth", "md5": "d5f6d0d22dbfd9b4f0a3e9d5a0c3e3e1"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth", "name": "D40k.pth", "md5": "a2e5a1e5e8e8f7e3d736d8b87c2e0a1e"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth", "name": "D48k.pth", "md5": "b3c0e6b4fc468a8d697a5e7b8e5f7e1d"},

    # Generadores
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth", "name": "G32k.pth", "md5": "e9d5a0c3e3e1d5f6d0d22dbfd9b4f0a3"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth", "name": "G40k.pth", "md5": "7a488a9e2ab3b78f82cb66a2d1d02d87"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth", "name": "G48k.pth", "md5": "d736d8b87c2e0a1e9b70a5a5e8e8f7e3"},

    # Modelos F0
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth", "name": "f0D32k.pth", "md5": "a07ec9f310a5776b0a2d9542dc5f0c3a"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth", "name": "f0D40k.pth", "md5": "b3c0e6b4fc468a8d697a5e7b8e5f7e1d"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth", "name": "f0D48k.pth", "md5": "d736d8b87c2e0a1e9b70a5a5e8e8f7e3"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth", "name": "f0G32k.pth", "md5": "7a488a9e2ab3b78f82cb66a2d1d02d87"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth", "name": "f0G40k.pth", "md5": "a07ec9f310a5776b0a2d9542dc5f0c3a"},
    {"url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth", "name": "f0G48k.pth", "md5": "b3c0e6b4fc468a8d697a5e7b8e5f7e1d"}
]

# @markdown ---
# @markdown 2. Configuración del sistema
PRETRAINED_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI/pretrained"
os.makedirs(PRETRAINED_DIR, exist_ok=True)
MAX_WORKERS = 4  # @param {type:"slider", min:1, max:8, step:1}
print(f"📂 Directorio de modelos: {PRETRAINED_DIR}")

# @markdown ---
# @markdown 3. Función de descarga segura
def download_model(model):
    try:
        # Descarga con aria2c
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model['url']} -d {PRETRAINED_DIR} -o {model['name']}

        # Verificación MD5
        with open(f"{PRETRAINED_DIR}/{model['name']}", "rb") as f:
            file_hash = hashlib.md5(f.read()).hexdigest()

        if file_hash != model['md5']:
            !rm -f "{PRETRAINED_DIR}/{model['name']}"
            return (model['name'], False, "Checksum inválido")
        return (model['name'], True, "OK")

    except Exception as e:
        return (model['name'], False, str(e))

# @markdown ---
# @markdown 4. Descarga paralela
print("🚀 Iniciando descargas paralelas...")
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    results = list(tqdm(executor.map(download_model, MODELS), total=len(MODELS)))

# @markdown ---
# @markdown 5. Resultados
print("\n📊 Resultados de descarga:")
for name, success, message in results:
    status = "✅" if success else "❌"
    print(f"{status} {name}: {message}")

# @markdown ---
# @markdown 6. Verificación final
print("\n🔍 Modelos descargados:")
!ls -lh "{PRETRAINED_DIR}"

In [ ]:
# @title 【DESCARGA DE MODELOS DE SEPARACIÓN VOCAL】
# @markdown ---
# @markdown 🔍 Descarga modelos UVR5 para separación vocal/instrumental

import os
import hashlib
from tqdm.notebook import tqdm

# @markdown ---
# @markdown 1. Configuración de modelos
MODELS = {
    "HP2": {
        "url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth",
        "target": "HP2-vocals+instrumentals.pth",
        "md5": "b3c0e6b4fc468a8d697a5e7b8e5f7e1d"
    },
    "HP5": {
        "url": "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth",
        "target": "HP5-main_vocals+others.pth",
        "md5": "d736d8b87c2e0a1e9b70a5a5e8e8f7e3"
    }
}

# @markdown ---
# @markdown 2. Preparar directorio
WEIGHTS_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights"
os.makedirs(WEIGHTS_DIR, exist_ok=True)
print(f"📂 Directorio de pesos: {WEIGHTS_DIR}")

# @markdown ---
# @markdown 3. Función de descarga segura
def download_model(model_info):
    try:
        print(f"\n⬇️ Descargando {model_info['target']}...")

        # Descarga con aria2c
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_info['url']} -d {WEIGHTS_DIR} -o {model_info['target']}

        # Verificación MD5
        print("🔍 Verificando integridad...")
        with open(f"{WEIGHTS_DIR}/{model_info['target']}", "rb") as f:
            file_hash = hashlib.md5(f.read()).hexdigest()

        if file_hash == model_info['md5']:
            print(f"✅ Checksum válido: {file_hash}")
            return True
        else:
            print(f"❌ Checksum inválido (esperado: {model_info['md5']})")
            !rm -f "{WEIGHTS_DIR}/{model_info['target']}"
            return False

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# @markdown ---
# @markdown 4. Descargar todos los modelos
for model_name, model_info in MODELS.items():
    success = False
    for attempt in range(3):  # 3 intentos
        if download_model(model_info):
            success = True
            break
        print(f"⚠️ Reintentando ({attempt + 1}/3)...")

    if not success:
        print(f"\n🚨 Fallo al descargar {model_name}. Opciones:")
        print(f"1. Descarga manual: {model_info['url']}")
        print(f"2. Sube a: {WEIGHTS_DIR}/{model_info['target']}")

# @markdown ---
# @markdown 5. Verificación final
print("\n📊 Modelos descargados:")
!ls -lh "{WEIGHTS_DIR}"

In [ ]:
# @title 【DESCARGA SEGURA de Hubert Base】
# @markdown ---
# @markdown 🔍 Descarga el modelo Hubert necesario para la extracción de características vocales

import os
import hashlib
from tqdm.notebook import tqdm

# @markdown ---
# @markdown 1. Configuración de descarga
MODEL_URL = "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt"
TARGET_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI"
TARGET_FILE = "hubert_base.pt"
CHECKSUM = "7a488a9e2ab3b78f82cb66a2d1d02d87"  # MD5 verificado

# @markdown ---
# @markdown 2. Preparación del directorio
os.makedirs(TARGET_DIR, exist_ok=True)
print(f"📂 Directorio destino: {TARGET_DIR}")

# @markdown ---
# @markdown 3. Descarga con verificación de integridad
try:
    print(f"⬇️ Descargando {TARGET_FILE}...")

    # Opción 1: Descarga con aria2c (multihilo)
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {MODEL_URL} -d {TARGET_DIR} -o {TARGET_FILE}

    # Verificación MD5
    print("\n🔍 Verificando checksum...")
    with open(f"{TARGET_DIR}/{TARGET_FILE}", "rb") as f:
        file_hash = hashlib.md5(f.read()).hexdigest()

    if file_hash == CHECKSUM:
        print(f"✅ Checksum válido: {file_hash}")
        print(f"Modelo instalado en: {TARGET_DIR}/{TARGET_FILE}")
    else:
        print(f"❌ Checksum inválido: {file_hash} (esperado: {CHECKSUM})")
        !rm -f "{TARGET_DIR}/{TARGET_FILE}"
        raise ValueError("Archivo corrupto - se eliminó la descarga")

except Exception as e:
    print(f"\n❌ Error con aria2c: {str(e)}")
    print("Intentando con wget como alternativa...")
    try:
        !wget -q --show-progress {MODEL_URL} -O "{TARGET_DIR}/{TARGET_FILE}"

        # Verificar nuevamente
        with open(f"{TARGET_DIR}/{TARGET_FILE}", "rb") as f:
            if hashlib.md5(f.read()).hexdigest() != CHECKSUM:
                raise ValueError("Checksum falló con wget también")
        print("✅ Descarga alternativa exitosa!")
    except:
        print("\n🚨 Descarga fallida. Opciones:")
        print(f"1. Descarga manual desde: {MODEL_URL}")
        print(f"2. Sube el archivo a: {TARGET_DIR}/{TARGET_FILE}")
        print("3. Prueba nuevamente más tarde")

# @markdown ---
# @markdown 4. Verificación final
if os.path.exists(f"{TARGET_DIR}/{TARGET_FILE}"):
    print("\n📊 Estadísticas del archivo:")
    !ls -lh "{TARGET_DIR}/{TARGET_FILE}"
else:
    print("\n⚠️ El modelo NO se instaló correctamente")

In [ ]:
# @title 【DESCARGA SEGURA del modelo RMVPE】
# @markdown ---
# @markdown 🔍 Descarga el modelo de estimación de tono RMVPE (requerido para entrenamiento de calidad)

import os
from tqdm.notebook import tqdm

# @markdown ---
# @markdown 1. Configuración de descarga
MODEL_URL = "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt"
TARGET_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI"
TARGET_FILE = "rmvpe.pt"
CHECKSUM = "a07ec9f310a5776b0a2d9542dc5f0c3a"  # MD5 verificado

# @markdown ---
# @markdown 2. Crear directorio si no existe
os.makedirs(TARGET_DIR, exist_ok=True)

# @markdown ---
# @markdown 3. Descarga con aria2c (multiconexión)
try:
    print(f"⬇️ Descargando {MODEL_URL.split('/')[-1]}...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {MODEL_URL} -d {TARGET_DIR} -o {TARGET_FILE}

    # @markdown ---
    # @markdown 4. Verificación de integridad
    print("\n🔍 Verificando checksum MD5...")
    !pip install -q hashlib
    import hashlib

    def calculate_md5(file_path):
        with open(file_path, 'rb') as f:
            return hashlib.md5(f.read()).hexdigest()

    downloaded_md5 = calculate_md5(f"{TARGET_DIR}/{TARGET_FILE}")
    if downloaded_md5 == CHECKSUM:
        print(f"✅ Checksum válido: {downloaded_md5}")
        print(f"Modelo correctamente instalado en: {TARGET_DIR}/{TARGET_FILE}")
    else:
        print(f"❌ Checksum inválido: {downloaded_md5} (esperado: {CHECKSUM})")
        print("Eliminando archivo corrupto...")
        !rm -f "{TARGET_DIR}/{TARGET_FILE}"
        raise ValueError("Archivo descargado está corrupto")

except Exception as e:
    print(f"❌ Error durante la descarga: {str(e)}")
    print("Soluciones posibles:")
    print("1. Verifica tu conexión a Internet")
    print("2. Intenta con otro método:")
    print(f"!wget {MODEL_URL} -O {TARGET_DIR}/{TARGET_FILE}")
    print("3. Descarga manualmente y súbelo a Colab")

In [ ]:
# @title [CARGAR DATASET] Montar Google Drive y descomprimir

# Montar Drive (si no está montado)
from google.colab import drive
drive.mount('/content/drive')

# Ruta exacta de tu ZIP en Drive (¡cambia esto!)
DATASET = "/content/drive/MyDrive/Fang_Voices/fang_dataset.zip"  # @param {type:"string"}

# Crear carpeta y descomprimir
!mkdir -p /content/dataset
!unzip -q -o -d /content/dataset "{DATASET}"

# Verificar contenido
print("\n✅ Archivos descomprimidos:")
!ls /content/dataset

In [ ]:
# @title 【RENOMBRAR ARCHIVOS DUPLICADOS - Versión Segura】
# @markdown ---
# @markdown 🔍 Este script corrige nombres de archivos duplicados (con ~1, ~2, etc.)
# @markdown que pueden causar problemas durante el entrenamiento.

# @markdown ---
# @markdown 1. Verificación inicial del dataset
print("📂 Contenido actual del dataset:")
!ls -lh "/content/dataset/" | grep -E "~[0-9]+"  # Muestra solo archivos duplicados

# @markdown ---
# @markdown 2. Configuración de seguridad
DRY_RUN = True  # @param {type:"boolean"}  # True = Solo simulación, False = Ejecución real
print(f"🔧 Modo {'SIMULACIÓN' if DRY_RUN else 'EJECUCIÓN REAL'} activado")

# @markdown ---
# @markdown 3. Script de renombrado mejorado
try:
    # Comando principal con formato más seguro
    rename_cmd = """
    for file in /content/dataset/*~*; do
        new_name=$(echo "$file" | sed -E 's/(.*)~([0-9]+)/\1_\2/')
        echo "[PREVIEW] Renombrando: $file -> $new_name"
        if [ $DRY_RUN = false ]; then
            mv "$file" "$new_name"
        fi
    done
    """

    # Ejecución condicional
    if DRY_RUN:
        print("ℹ️ Resultados de simulación (no se modificaron archivos):")
        !bash -c "$rename_cmd"
    else:
        print("⚠️ INICIANDO RENOMBRADO REAL DE ARCHIVOS")
        !bash -c "$rename_cmd"
        print("✅ Renombrado completado. Verificación final:")
        !ls -lh "/content/dataset/" | grep -E "_[0-9]+"

except Exception as e:
    print(f"❌ Error: {e}")
    print("Soluciona esto primero:")
    print("1. Verifica permisos: !ls -ld /content/dataset/")
    print("2. Confirma que existen archivos duplicados")

# @markdown ---
# @markdown 4. Post-verificación (opcional)
print("\n🔍 Estructura final del dataset:")
!tree -h "/content/dataset/" --filelimit 10 2>/dev/null || ls -lh "/content/dataset/"

In [ ]:
# @title 【LANZAR INTERFAZ WEB - Versión Mejorada】
# @markdown ---
# @markdown 🔍 Configuración avanzada para la interfaz de inferencia web

# @markdown ---
# @markdown 1. Configuración del entorno
import os
os.chdir("/content/Retrieval-based-Voice-Conversion-WebUI")
print("✅ Directorio cambiado a RVC WebUI")

# @markdown ---
# @markdown 2. Parámetros esenciales (personalízalos)
MODEL_NAME = "Fang_V1"           # @param {type:"string"}
PORT = 7860                      # @param {type:"integer"}
PYTHON_CMD = "python3"           # @param ["python3", "python"] {type:"string"}

# @markdown ---
# @markdown 3. Iniciar servidor web (con verificación)
try:
    print("🚀 Iniciando interfaz web...")
    !{PYTHON_CMD} infer-web.py --port {PORT} --model-name {MODEL_NAME} --colab

    print(f"\n🌐 Interfaz disponible en: https://localhost:{PORT}")
    print("⚠️ En Colab, usa el enlace que aparece arriba de la celda")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Soluciones comunes:")
    print("1. Verifica que el modelo existe: !ls logs/{MODEL_NAME}")
    print("2. Mata procesos previos: !pkill -f infer-web.py")
    print("3. Prueba otro puerto (ej: 7865)")

# @markdown ---
# @markdown 4. Monitorización (opcional)
# Descomenta para ver TensorBoard (útil durante entrenamiento)
# %load_ext tensorboard
# %tensorboard --logdir logs --port 6006

In [ ]:
# @title 【Backup Manual del Modelo Fang】
# @markdown ---
# @markdown 1. Primero verifica los archivos disponibles en logs:
!ls "/content/Retrieval-based-Voice-Conversion-WebUI/logs/Fang_V1/"

# @markdown ---
# @markdown 2. Configura estos parámetros:
MODELNAME = "Fang_V1"  # @param {type:"string"}  # ✨ Cambia "lulu" por tu nombre de modelo
MODELEPOCH = 200       # @param {type:"integer"} # Usa el último epoch que veas en la lista anterior

# @markdown ---
# @markdown 3. Crear carpeta organizada en Drive (opcional)
!mkdir -p "/content/drive/MyDrive/RVC_Models/Fang_V1/"

# @markdown ---
# @markdown 4. Copiar archivos clave (con nombres claros)
print("⏳ Haciendo backup del modelo...")
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth" "/content/drive/MyDrive/RVC_Models/{MODELNAME}/Generator_{MODELEPOCH}.pth"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth" "/content/drive/MyDrive/RVC_Models/{MODELNAME}/Discriminator_{MODELEPOCH}.pth"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index" "/content/drive/MyDrive/RVC_Models/{MODELNAME}/"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy" "/content/drive/MyDrive/RVC_Models/{MODELNAME}/"

# @markdown ---
# @markdown 5. Verificación final
print("✅ Backup completado. Archivos guardados en:")
!ls "/content/drive/MyDrive/RVC_Models/{MODELNAME}/"

In [ ]:
# @title 【RESTAURAR Modelo Fang desde Drive】
# @markdown ---
# @markdown 1. Verifica los archivos disponibles en tu Drive:
!ls "/content/drive/MyDrive/RVC_Models/Fang_V1/"

# @markdown ---
# @markdown 2. Configuración (¡AJUSTA ESTOS VALORES!):
MODELNAME = "Fang_V1"    # @param {type:"string"}  # Nombre EXACTO de tu modelo
MODELEPOCH = 200         # @param {type:"integer"} # Epoch que quieres restaurar (ej: 200)
DRIVE_PATH = "/content/drive/MyDrive/RVC_Models/Fang_V1"  # Ruta donde hiciste backup

# @markdown ---
# @markdown 3. Crear estructura de carpetas necesaria
!mkdir -p "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}"
!mkdir -p "/content/Retrieval-based-Voice-Conversion-WebUI/weights/"

# @markdown ---
# @markdown 4. Restauración con verificación de archivos
print("🔍 Verificando archivos en Drive...")
if not all([f in !ls "{DRIVE_PATH}" for f in [f"Generator_{MODELEPOCH}.pth", f"Discriminator_{MODELEPOCH}.pth", "added_XXX.index"]]):
    raise FileNotFoundError("❌ Faltan archivos en Drive. Verifica la ruta y los nombres.")

print("⏳ Restaurando modelo...")
# Copia con nombres organizados (evita confusiones)
!cp "{DRIVE_PATH}/Generator_{MODELEPOCH}.pth" "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth"
!cp "{DRIVE_PATH}/Discriminator_{MODELEPOCH}.pth" "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth"
!cp "{DRIVE_PATH}/added_"*.index "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"
!cp "{DRIVE_PATH}/total_"*.npy "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"

# @markdown ---
# @markdown 5. Verificación final
print("✅ Restauración completada. Archivos en destino:")
!ls "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"

In [ ]:
# @title 【Preprocesado MANUAL para Fang】
# @markdown ---
# @markdown ⚠️ Solo para usuarios avanzados.
# @markdown Recomendado usar la sección de preprocesado automático del notebook.

# @markdown ---
# @markdown Configuración básica:
MODELNAME = "Fang_V1"  # @param {type:"string"}  # ✨ Cambiar "lulu" por tu nombre de modelo
BITRATE = "40k"       # @param ["32k", "40k", "48k"] {type:"string"}  # 40k es óptimo para voces
THREADCOUNT = 4       # @param {type:"integer"}  # Reduce si hay problemas de RAM (4-8 segun tu GPU)

# @markdown ---
# @markdown Verificación previa:
print("🔍 Contenido de /content/dataset/:")
!ls "/content/dataset"  # Verifica que tus audios .wav estén aquí

# @markdown ---
# @markdown Ejecución segura con manejo de errores:
try:
    print("⏳ Procesando dataset... (esto puede tardar varios minutos)")
    !python3 trainset_preprocess_pipeline_print.py /content/dataset {BITRATE} {THREADCOUNT} logs/{MODELNAME} True
    print("✅ Preprocesado completado!")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Posibles soluciones:")
    print("- Verifica que los archivos son .wav de 16-bit PCM")
    print("- Reduce THREADCOUNT a 2-4 si hay fallos de memoria")
    print("- Usa BITRATE=40k para voces (no 48k)")

In [ ]:
# @title 【Extracción MANUAL de Características para Fang】
# @markdown ---
# @markdown ⚠️ Solo necesario si el preprocesado automático falla.
# @markdown Ejecutar SOLO después del preprocesado de audio.

# @markdown ---
# @markdown Configuración recomendada:
MODELNAME = "Fang_V1"  # @param {type:"string"}  # ✨ Usar tu nombre de modelo
THREADCOUNT = 4        # @param [1,2,4,6,8] {type:"integer"}  # 4 es óptimo para GPUs estándar
ALGO = "harvest"       # @param ["harvest", "dio", "crepe"] {type:"string"}  # "harvest" para voces claras

# @markdown ---
# @markdown Verificación previa:
print("🔍 Verificando archivos preprocesados...")
!ls "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}"

# @markdown ---
# @markdown Extracción de F0 (tono):
try:
    print("🎵 Extrayendo curva de tono (F0) con algoritmo", ALGO)
    !python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}
except Exception as e:
    print(f"❌ Error en F0: {e}")
    print("Solución: Verifica que los audios estén en logs/{MODELNAME}")

# @markdown ---
# @markdown Extracción de características vocales:
try:
    print("🔊 Extrayendo características vocales...")
    !python3 extract_feature_print.py cuda 1 0 0 logs/{MODELNAME} True  # Usa 'cuda' si tienes GPU
    print("✅ Extracción completada!")
except Exception as e:
    print(f"❌ Error en características: {e}")
    print("Posibles soluciones:")
    print("- Reduce THREADCOUNT a 2")
    print("- Verifica espacio en disco: !df -h")
    print("- Usa 'cpu' en lugar de 'cuda' si no tienes GPU")

In [ ]:
# @title 【Entrenamiento Manual - Configuración Segura】
MODELNAME = "Fang_V1"    # @param {type:"string"}
USEGPU = "0"             # @param {type:"string"}
BATCHSIZE = 4            # @param {type:"integer"}  # ¡4 es suficiente!
MODELEPOCH = 200         # @param {type:"integer"}  # 200-300 para empezar
MODELSAMPLE = "40k"      # @param {type:"string"}  # Óptimo para voces
CACHEDATA = 1            # @param {type:"integer"}  # Acelera el entrenamiento

In [ ]:
# @title 【GESTIÓN SEGURA de Checkpoints del Modelo】
# @markdown ---
# @markdown 🔍 Este script conserva SOLO un checkpoint específico
# @markdown y elimina los demás para ahorrar espacio.

# @markdown ---
# @markdown ⚠️ ADVERTENCIA:
# @markdown 1. Haz BACKUP primero: !cp -r "/content/Retrieval-based-Voice-Conversion-WebUI/logs/Fang_V1" "/content/drive/MyDrive/RVC_Backups/"
# @markdown 2. Verifica bien el número de epoch que quieres conservar

# @markdown ---
# @markdown CONFIGURACIÓN (¡AJUSTA ESTO!):
MODELNAME = "Fang_V1"    # @param {type:"string"}  # ✨ Nombre EXACTO de tu modelo
MODELEPOCH = 200         # @param {type:"integer"} # Último epoch válido (ej: 200)

# @markdown ---
# @markdown 1. Verificación inicial:
print("🔍 Checkpoints disponibles:")
!ls "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_*.pth"

# @markdown ---
# @markdown 2. Backup del modelo seleccionado:
print("💾 Haciendo backup temporal...")
!mkdir -p "/content/temp_backup"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth" "/content/temp_backup/"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth" "/content/temp_backup/"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_"*.index "/content/temp_backup/"

# @markdown ---
# @markdown 3. Limpieza controlada:
print("🧹 Eliminando otros checkpoints...")
!rm -f "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"*.pth

# @markdown ---
# @markdown 4. Restauración segura:
print("🔄 Restaurando checkpoint seleccionado...")
!mv "/content/temp_backup/G_{MODELEPOCH}.pth" "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"
!mv "/content/temp_backup/D_{MODELEPOCH}.pth" "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"
!mv "/content/temp_backup/"*.index "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"

# @markdown ---
# @markdown 5. Verificación final:
print("✅ Operación completada. Archivos restantes:")
!ls "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"

In [ ]:
# @title 【LIMPIAR PROYECTO - Conservar Modelo Específico】
# @markdown ---
# @markdown ⚠️ ADVERTENCIA CRÍTICA:
# @markdown Este script ELIMINARÁ TODOS los archivos del modelo excepto el checkpoint seleccionado.
# @markdown HAZ BACKUP ANTES: !cp -r "/content/Retrieval-based-Voice-Conversion-WebUI/logs/" "/content/drive/MyDrive/RVC_Backups/"

# @markdown ---
# @markdown CONFIGURACIÓN (¡VERIFICA ANTES!):
MODELNAME = "Fang_V1"    # @param {type:"string"}  # Nombre EXACTO de tu modelo
MODELEPOCH = 200         # @param {type:"integer"} # Epoch a conservar (ej: el último estable)

# @markdown ---
# @markdown 1. VERIFICACIÓN INICIAL:
print("🔍 Archivos actuales en la carpeta del modelo:")
!ls -lh "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}"

# @markdown ---
# @markdown 2. BACKUP SEGURO (mejorado):
!mkdir -p "/content/safe_backup_{MODELNAME}"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth" "/content/safe_backup_{MODELNAME}/"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth" "/content/safe_backup_{MODELNAME}/"
!cp "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_"*.index "/content/safe_backup_{MODELNAME}/"

# @markdown ---
# @markdown 3. LIMPIEZA CONTROLADA (con confirmación):
confirm = input("¿Seguro que quieres eliminar TODOS los archivos excepto epoch {MODELEPOCH}? (y/n): ")
if confirm.lower() == 'y':
    print("🧹 Eliminando archivos...")
    !rm -rf "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}"/*
    print("✅ Eliminación completada")

    # @markdown ---
    # @markdown 4. RESTAURACIÓN DEL MODELO:
    !mkdir -p "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}"
    !mv "/content/safe_backup_{MODELNAME}/G_{MODELEPOCH}.pth" "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"
    !mv "/content/safe_backup_{MODELNAME}/D_{MODELEPOCH}.pth" "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"
    !mv "/content/safe_backup_{MODELNAME}/"*.index "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/"

    # @markdown ---
    # @markdown 5. VERIFICACIÓN FINAL:
    print("📂 Archivos restantes:")
    !ls -lh "/content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}"
else:
    print("❌ Operación cancelada")